# Inference EfficientNetV2 Tipe B2

In [ ]:
# import os
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.applications import EfficientNetV2B2
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from sklearn.metrics import average_precision_score, f1_score
# from sklearn.model_selection import train_test_split
# from helper_functions import create_tensorboard_callback
# import time
# import numpy as np

In [ ]:
# def load_and_preprocess_data(csv_path, image_folder, img_size, batch_size):
#     # Read CSV
#     try:
#         df_merged = pd.read_csv(csv_path)
#     except Exception as e:
#         print(f"Error reading CSV file: {e}")
#         return None, None, None

#     if df_merged.empty or 'filename' not in df_merged.columns:
#         print("CSV file is empty or does not contain the 'filename' column")
#         return None, None, None

#     # Shuffle the DataFrame and limit it to the first 2000 rows
#     df_merged = df_merged.sample(frac=1, random_state=42).head(2000)
#     df_merged['filename'] = df_merged['filename'].apply(lambda x: os.path.join(image_folder, x + ".png"))

#     # Split dataset
#     train_df, test_df = train_test_split(df_merged, test_size=0.2, random_state=42)
#     test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

#     # Data generators
#     train_datagen = ImageDataGenerator(
#         rescale=1.0/255,
#         rotation_range=20,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest'
#     )
#     validation_datagen = ImageDataGenerator(rescale=1.0/255)

#     try:
#         train_generator = train_datagen.flow_from_dataframe(
#             dataframe=train_df,
#             x_col='filename',
#             y_col=train_df.columns[1:],
#             target_size=(img_size, img_size),
#             batch_size=batch_size,
#             class_mode='raw'
#         )
#         validation_generator = validation_datagen.flow_from_dataframe(
#             dataframe=val_df,
#             x_col='filename',
#             y_col=val_df.columns[1:],
#             target_size=(img_size, img_size),
#             batch_size=batch_size,
#             class_mode='raw'
#         )
#         test_generator = validation_datagen.flow_from_dataframe(
#             dataframe=test_df,
#             x_col='filename',
#             y_col=test_df.columns[1:],
#             target_size=(img_size, img_size),
#             batch_size=batch_size,
#             class_mode='raw',
#             shuffle=False
#         )
#     except Exception as e:
#         print(f"Error creating data generators: {e}")
#         return None, None, None

#     return train_generator, validation_generator, test_generator

# def create_model(img_size, num_classes, base_weights_path):
#     base_model = EfficientNetV2B2(weights=base_weights_path, include_top=False, input_tensor=Input(shape=(img_size, img_size, 3)))
#     base_model.trainable = False

#     inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3), name='Input_layer')
#     x = base_model(inputs, training=False)
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

#     model = tf.keras.Model(inputs, outputs)
#     return model

In [1]:
# def evaluate_model(model, test_generator):
#     start_time = time.time()
#     test_loss, test_accuracy = model.evaluate(test_generator)
#     end_time = time.time()

#     inference_time = (end_time - start_time) / len(test_generator)

#     # Get true labels and predictions
#     y_true = test_generator.labels
#     y_pred = model.predict(test_generator)
    
#     # Calculate mAP 0.5 and mAP 0.5:0.95
#     average_precisions = []
#     for i in range(y_true.shape[1]):
#         average_precisions.append(average_precision_score(y_true[:, i], y_pred[:, i]))
#     mAP_0_5 = np.mean(average_precisions)
    
#     # Convert predictions to binary
#     y_pred_binary = (y_pred > 0.5).astype(int)
    
#     # Calculate F1-Score
#     f1 = f1_score(y_true, y_pred_binary, average='macro')

#     print(f"Test Loss: {test_loss}")
#     print(f"Test Accuracy: {test_accuracy}")
#     print(f"mAP 0.5: {mAP_0_5}")
#     print(f"F1-Score: {f1}")
#     print(f"Inference Time (per image): {inference_time} seconds")

#     return mAP_0_5, f1, inference_time

In [11]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score, f1_score
import time
import numpy as np

def load_and_preprocess_data(csv_path, image_folder, img_size, batch_size):
    # Read CSV
    try:
        df_merged = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

    if df_merged.empty or 'filename' not in df_merged.columns:
        print("CSV file is empty or does not contain the 'filename' column")
        return None

    # Shuffle the DataFrame and limit it to the first 2000 rows
    df_merged = df_merged.sample(frac=1, random_state=42).head(2000)
    df_merged['filename'] = df_merged['filename'].apply(lambda x: os.path.join(image_folder, x + ".png"))

    # Split dataset
    _, test_df = train_test_split(df_merged, test_size=0.2, random_state=42)
    test_df, _ = train_test_split(test_df, test_size=0.5, random_state=42)

    # Data generator
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    try:
        test_generator = validation_datagen.flow_from_dataframe(
            dataframe=test_df,
            x_col='filename',
            y_col=test_df.columns[1:],
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='raw',
            shuffle=False
        )
    except Exception as e:
        print(f"Error creating data generators: {e}")
        return None

    return test_generator

def create_model(img_size, num_classes):
    base_model = tf.keras.applications.EfficientNetV2B2(include_top=False, input_shape=(img_size, img_size, 3))
    base_model.trainable = False

    inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3), name='Input_layer')
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    initial_learning_rate = 1e-4

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=1.0,
        staircase=True)

    model = tf.keras.Model(inputs, outputs)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

def calculate_map(y_true, y_pred, thresholds):
    average_precisions = []
    for i in range(y_true.shape[1]):
        aps = [average_precision_score(y_true[:, i], (y_pred[:, i] > t).astype(int)) for t in thresholds]
        average_precisions.append(np.mean(aps))
    return np.mean(average_precisions)

def evaluate_model(model, checkpoint_path, test_generator):
    # Load the saved weights
    model.load_weights(checkpoint_path)
    
    start_time = time.time()
    test_loss, test_accuracy = model.evaluate(test_generator)
    end_time = time.time()

    inference_time = (end_time - start_time) / len(test_generator)

    # Get true labels and predictions
    y_true = test_generator.labels
    y_pred = model.predict(test_generator)
    
    # Calculate mAP 0.5
    mAP_0_5 = calculate_map(y_true, y_pred, [0.5])
    
    # Calculate mAP 0.5:0.95
    mAP_0_5_0_95 = calculate_map(y_true, y_pred, np.arange(0.5, 1.0, 0.05))
    
    # Convert predictions to binary
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    # Calculate F1-Score
    f1 = f1_score(y_true, y_pred_binary, average='macro')

    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")
    print(f"mAP 0.5: {mAP_0_5}")
    print(f"mAP 0.5:0.95: {mAP_0_5_0_95}")
    print(f"F1-Score: {f1}")
    print(f"Inference Time (per image): {inference_time} seconds")

    return mAP_0_5, mAP_0_5_0_95, f1, inference_time

# if __name__ == "__main__":
csv_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/image_datasets.csv"  # Update with the actual path
image_folder = "D:/Skripsweet/BISMILLAH_KELAR/dataset/images"
img_size = 640
batch_size = 10
num_classes = 24

test_generator = load_and_preprocess_data(csv_path, image_folder, img_size, batch_size)

if test_generator:
    model = create_model(img_size, num_classes)
    checkpoint_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/runs/best_top_layers.ckpt"  # Path to your best model checkpoint files (index and data-0000-of-0001)
    mAP_0_5, mAP_0_5_0_95, f1, inference_time = evaluate_model(model, checkpoint_path, test_generator)


Found 200 validated image filenames.
20/20 [==============================] - 43s 2s/step
Test Loss: 0.6916007995605469
Test Accuracy: 0.5333333611488342
mAP 0.5: 0.49870922157158754
mAP 0.5:0.95: 0.4998551042559242
F1-Score: 0.38171091390450124
Inference Time (per image): 2.626672101020813 seconds


c:\Users\miran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
# if __name__ == "__main__":
csv_path_binary = "D:/Skripsweet/BISMILLAH_KELAR/dataset/datasets_binarize.csv"  # Update with the actual path
image_folder_binary = "D:/Skripsweet/BISMILLAH_KELAR/dataset/binary_images"
img_size = 640
batch_size = 10
num_classes = 24

test_generator = load_and_preprocess_data(csv_path_binary, image_folder_binary, img_size, batch_size)

if test_generator:
    model = create_model(img_size, num_classes)
    checkpoint_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/runs/best_top_layers.ckpt"  # Path to your best model checkpoint files (index and data-0000-of-0001)
    mAP_0_5, mAP_0_5_0_95, f1, inference_time = evaluate_model(model, checkpoint_path, test_generator)

Found 200 validated image filenames.
20/20 [==============================] - 11s 447ms/step
Test Loss: 0.6930428147315979
Test Accuracy: 0.5247916579246521
mAP 0.5: 0.5018398288583915
mAP 0.5:0.95: 0.5001839828858392
F1-Score: 0.43899973296792627
Inference Time (per image): 1.4475922107696533 seconds


c:\Users\miran\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
csv_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/image_datasets.csv"  # Update with the actual path
image_folder = "D:/Skripsweet/BISMILLAH_KELAR/dataset/images"
img_size = 640
batch_size = 10
num_classes = 24

test_generator = load_and_preprocess_data(csv_path, image_folder, img_size, batch_size)

if test_generator:
    model = create_model(img_size, num_classes)
    # checkpoint_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/runs/best_top_layers.ckpt"  # Path to your best model checkpoint files (index and data-0000-of-0001)
    checkpoint_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/runs/best_all_layers.ckpt"
    mAP_0_5, mAP_0_5_0_95, f1, inference_time = evaluate_model(model, checkpoint_path, test_generator)

Found 200 validated image filenames.


20/20 [==============================] - 23s 1s/step
Test Loss: 0.6933253407478333
Test Accuracy: 0.4633333384990692
mAP 0.5: 0.5000835905349795
mAP 0.5:0.95: 0.5000083590534979
F1-Score: 0.34795729659869895
Inference Time (per image): 2.090709412097931 seconds


In [14]:
csv_path_binary = "D:/Skripsweet/BISMILLAH_KELAR/dataset/datasets_binarize.csv"  # Update with the actual path
image_folder_binary = "D:/Skripsweet/BISMILLAH_KELAR/dataset/binary_images"
img_size = 640
batch_size = 10
num_classes = 24

test_generator = load_and_preprocess_data(csv_path_binary, image_folder_binary, img_size, batch_size)

if test_generator:
    model = create_model(img_size, num_classes)
    # checkpoint_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/runs/best_top_layers.ckpt"  # Path to your best model checkpoint files (index and data-0000-of-0001)
    checkpoint_path = "D:/Skripsweet/BISMILLAH_KELAR/dataset/runs/best_all_layers.ckpt"
    mAP_0_5, mAP_0_5_0_95, f1, inference_time = evaluate_model(model, checkpoint_path, test_generator)

Found 200 validated image filenames.


20/20 [==============================] - 11s 435ms/step
Test Loss: 0.6933237314224243
Test Accuracy: 0.4658333361148834
mAP 0.5: 0.5008132573159579
mAP 0.5:0.95: 0.5000813257315958
F1-Score: 0.36434213174424696
Inference Time (per image): 1.0774729013442994 seconds


In [15]:
def load_and_preprocess_data(csv_path, image_folder, img_size, batch_size):
    # Read CSV
    try:
        df_merged = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None, None, None

    if df_merged.empty or 'filename' not in df_merged.columns:
        print("CSV file is empty or does not contain the 'filename' column")
        return None, None, None

    # Shuffle the DataFrame and limit it to the first 2000 rows
    df_merged = df_merged.sample(frac=1, random_state=42).head(2000)
    df_merged['filename'] = df_merged['filename'].apply(lambda x: os.path.join(image_folder, x + ".png"))

    # Split dataset
    train_df, test_df = train_test_split(df_merged, test_size=0.2, random_state=42)
    test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)

    # Data generators
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    try:
        train_generator = train_datagen.flow_from_dataframe(
            dataframe=train_df,
            x_col='filename',
            y_col=train_df.columns[1:],
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='raw'
        )
        validation_generator = validation_datagen.flow_from_dataframe(
            dataframe=val_df,
            x_col='filename',
            y_col=val_df.columns[1:],
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='raw'
        )
        test_generator = validation_datagen.flow_from_dataframe(
            dataframe=test_df,
            x_col='filename',
            y_col=test_df.columns[1:],
            target_size=(img_size, img_size),
            batch_size=batch_size,
            class_mode='raw',
            shuffle=False
        )
    except Exception as e:
        print(f"Error creating data generators: {e}")
        return None, None, None

    return train_generator, validation_generator, test_generator

In [21]:
def create_model_1(img_size, num_classes, base_weights_path):
    base_model = tf.keras.applications.EfficientNetV2B2(include_top=False, input_shape=(img_size, img_size, 3))
    base_model.trainable = False

    inputs = tf.keras.layers.Input(shape=(img_size, img_size, 3), name='Input_layer')
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    initial_learning_rate = 1e-4

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=1.0,
        staircase=True)

    model = tf.keras.Model(inputs, outputs)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

In [36]:
def train_all_layers(model, initial_learning_rate):
    model.layers[1].trainable = True
    for layer in model.layers[1].layers[:-60]:
        layer.trainable = False

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=1.0,
        staircase=True)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

In [35]:
def train_top_layers(model, initial_learning_rate):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=1.0,
        staircase=True)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

In [24]:
train_generator, validation_generator, test_generator = load_and_preprocess_data(csv_path, image_folder, img_size, batch_size)
    
num_classes = 24
base_weights_path = r"D:/Skripsweet/BISMILLAH_KELAR/efficientnetv2-b2_notop.h5"
model = create_model_1(img_size, num_classes, base_weights_path)

Found 1600 validated image filenames.
Found 200 validated image filenames.
Found 200 validated image filenames.


In [38]:
no_weights = train_top_layers(model, 1e-4)
no_weights.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 640, 640, 3)]     0         
                                                                 
 efficientnetv2-b2 (Function  (None, 20, 20, 1408)     8769374   
 al)                                                             
                                                                 
 global_average_pooling2d_7   (None, 1408)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_7 (Dense)             (None, 24)                33816     
                                                                 
Total params: 8,803,190
Trainable params: 2,735,016
Non-trainable params: 6,068,174
_________________________________________________________________


In [39]:
with_weights = train_all_layers(model, 1e-4)
with_weights.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 640, 640, 3)]     0         
                                                                 
 efficientnetv2-b2 (Function  (None, 20, 20, 1408)     8769374   
 al)                                                             
                                                                 
 global_average_pooling2d_7   (None, 1408)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_7 (Dense)             (None, 24)                33816     
                                                                 
Total params: 8,803,190
Trainable params: 2,735,016
Non-trainable params: 6,068,174
_________________________________________________________________


# Inference EFFNET

In [1]:
image_path_color = "D:/Skripsweet/BISMILLAH_KELAR/datasets/images/val/13143_lr-0.png"
image_path_binarize = "D:/Skripsweet/BISMILLAH_KELAR/datasets_binarize/val/images/13143_lr-0_1.png"

In [8]:
import numpy as np
from keras.utils import load_img, img_to_array

img = load_img(image_path_color, target_size=(640, 640)) # Replace img_width, img_height with your model's expected input size
img_array_color = img_to_array(img)
img_array_color = np.expand_dims(img_array_color, axis=0)  # Add batch dimension
img_array_color = img_array_color / 255.0  # Normalize pixel values (if needed)


In [12]:
img_binary = load_img(image_path_binarize, target_size=(640, 640)) # Replace img_width, img_height with your model's expected input size
img_array_binary = img_to_array(img_binary)
img_array_binary = np.expand_dims(img_array_binary, axis=0)  # Add batch dimension
img_array_binary = img_array_binary / 255.0  # Normalize pixel values (if needed)

In [4]:
# Class labels for display
class_names = ['I arrhythmia', 'II arrhythmia', 'III arrhythmia', 'aVF arrhythmia', 'aVL arrhythmia', 'aVR arrhythmia',
                'V1 arrhythmia', 'V2 arrhythmia', 'V3 arrhythmia', 'V4 arrhythmia', 'V5 arrhythmia', 'V6 arrhythmia']

## Color

In [20]:
import pandas as pd
from tensorflow import keras

# 1. Load the .h5 Model For EffNetV2 type B2 and No Pre-Trained Weights
model_path_color_no_weights = 'D:/Skripsweet/BISMILLAH_KELAR/dataset/early_model_ori.h5'  # Replace with your actual path
model_color_no_weights = keras.models.load_model(model_path_color_no_weights)

# Make Predictions
predictions_1 = model_color_no_weights.predict(img_array_color)

print(predictions_1)

1/1 [==============================] - 2s 2s/step
[[0.4759252  0.47725397 0.47613168 0.47663113 0.476132   0.47583202
  0.47584096 0.47579756 0.4757403  0.4754664  0.47685066 0.47542414
  0.525833   0.5258463  0.52203983 0.52417153 0.52350175 0.52321243
  0.5252535  0.52343076 0.52427536 0.52412486 0.5228706  0.524206  ]]


In [21]:
import pandas as pd
from tensorflow import keras

# 1. Load the .h5 Model For EffNetV2 type B2 and No Pre-Trained Weights
model_path_color_weights = 'D:/Skripsweet/BISMILLAH_KELAR/dataset/final_model_ori.h5'  # Replace with your actual path
model_color_weights = keras.models.load_model(model_path_color_weights)

# Make Predictions
predictions_2 = model_color_weights.predict(img_array_color)

print(predictions_2)

1/1 [==============================] - 2s 2s/step
[[0.4909087  0.4904405  0.49054766 0.4897278  0.49056837 0.49050903
  0.49063584 0.49047273 0.49079096 0.4901032  0.4897929  0.490056
  0.5095559  0.50909764 0.5101658  0.5100444  0.5096905  0.5094981
  0.50969005 0.5094849  0.509218   0.50902426 0.50962055 0.50987536]]


## Binary

In [18]:
import pandas as pd
from tensorflow import keras

# 1. Load the .h5 Model For EffNetV2 type B2 and No Pre-Trained Weights
model_path_binary_no_weights = 'D:/Skripsweet/BISMILLAH_KELAR/dataset/early_model_ori_binary.h5'  # Replace with your actual path
model_binary_no_weights = keras.models.load_model(model_path_binary_no_weights)

# Make Predictions
predictions_3 = model_binary_no_weights.predict(img_array_binary)

print(predictions_3)

1/1 [==============================] - 2s 2s/step
[[0.46322677 0.462527   0.46235844 0.46241063 0.46240157 0.4625566
  0.46225277 0.4625731  0.46229872 0.46240702 0.4619704  0.46293682
  0.5368149  0.5377579  0.5382155  0.53768986 0.5374766  0.5368958
  0.5378943  0.5382453  0.5373464  0.5366147  0.53751844 0.53716826]]


In [19]:
import pandas as pd
from tensorflow import keras

# 1. Load the .h5 Model For EffNetV2 type B2 and No Pre-Trained Weights
model_path_binary_weights = 'D:/Skripsweet/BISMILLAH_KELAR/dataset/final_model_ori_binary.h5'  # Replace with your actual path
model_binary_weights = keras.models.load_model(model_path_binary_weights)

# Make Predictions
predictions_4 = model_binary_weights.predict(img_array_binary)

print(predictions_4)

1/1 [==============================] - 3s 3s/step
[[0.48997414 0.49017546 0.48989487 0.49006242 0.4896534  0.49006423
  0.48983675 0.490274   0.48963267 0.49004924 0.4900495  0.4897588
  0.5101237  0.5098772  0.51021725 0.50992167 0.51035124 0.5104474
  0.51025385 0.5103281  0.5103459  0.51019084 0.510339   0.5098905 ]]
